In [1]:
import nltk
#Необходимо прогнать эти строки при первом запуске
#nltk.download('stopwords')
#nltk.download('punkt')
#nltk.download('wordnet')

import pymorphy2
import codecs
import string
import re
import math

from bs4 import BeautifulSoup
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
analyzer = pymorphy2.MorphAnalyzer()
        
rus_filter = re.compile("[а-яА-Я]+")
eng_filter = re.compile("[a-zA-Z]+")



# список символов, которые удаются из текста
MARKS = [',', '.', ':', '?', '«', '»', '-', '(', ')', '!', '\'', "—", ';', "”", "...", "\'\'", "/**//**/",
         "“", "„", "–"]

In [2]:
def parse_words_from_html(file):
    html_page = codecs.open(f"D:/Documents/GitHub/WebParser/src/pages/{file}", 'r', 'utf-8')
    
    soup = BeautifulSoup(html_page.read(), features='html.parser')
    # kill all script, style, meta, links, span, a, time, button, li, dt, h2, h3, legend elements
    for script in soup(
            ["script", "style", "meta", "link", "span", "a", "time", "button", "li", "dt", "h2", "h3", "legend"]):
        script.extract()  # rip it out

    # get text
    text = soup.get_text()
    # break into lines and remove leading and trailing space on each
    lines = (line.strip() for line in text.splitlines())
    # break multi-headlines into a line each
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    # drop blank lines
    text = '\n'.join(chunk for chunk in chunks if chunk)
    # убираем знаки препинания
    pattern = r"[^\w]"
    text = re.sub(pattern, " ", text)
    # split to words
    return text.split()

def tokenize():
    words = []
    docs_dict = {}
    with open("D:/Documents/GitHub/WebParser/src/pages/index.txt", "r") as index:
        lines = index.readlines()
        numbers = [line[: line.find(" ")] for line in lines]
        for num in numbers:
            words.extend([[parse_words_from_html(f"{num}.txt"), num]])
            docs_dict[int(num)] = 0
        with open("words.txt", "w", encoding="utf-8") as file:
            for item in words:
                filtered_words = filter_words(item[0])
                for word in filtered_words:
                    docs_dict[int(item[1])] += 1
                    file.write(word + ' ' + item[1] + '\n')
    return docs_dict
                
def filter_words(words):
    words = list(filter(lambda word: (word not in string.punctuation) and (word not in MARKS), words))
    filtered_words = []
    for word in words:
        filtered_words.append("".join(filter(lambda char: char not in MARKS, word)))
    stop_words_rus = set(stopwords.words('russian'))
    stop_words_eng = set(stopwords.words('english'))
    filtered_words = [w.lower() for w in filtered_words if w not in stop_words_rus]
    filtered_words = [w.lower() for w in filtered_words if w not in stop_words_eng]
    filtered_words = filter(lambda word: word.isalpha(), filtered_words)
    return filtered_words

def lemmatize(docs_dict):
    with open("words.txt", "r", encoding="utf-8") as words_file:
        lines = words_file.readlines()
        
    rus_lines = [line for line in filter(rus_filter.match, lines)]
    eng_lines = [line for line in filter(eng_filter.match, lines)]
    
    rus_words = []
    eng_words = []
    
    for line in rus_lines:
        rus_words.append(line.strip().split())
    for line in eng_lines:
        eng_words.append(line.strip().split())
    
    lem_dict = {}
    index_dict = {}
    token_count_dict = {}
    #print(rus_words)
    #print(eng_words)
    
    for word in rus_words:
        # получаем начальную форму слова
        normal_form = get_normal_form_rus(word[0])
        if normal_form:
            # если такое слово еще не встречалось,создаем ключ с нормальной формой и помещаем само слово как значение
            if normal_form not in lem_dict.keys():
                lem_dict[normal_form] = [word[0].strip()]
                index_dict[normal_form] = [int(word[1])]
            # если такое слово встречалось ранее, добавляем его вариацию в список значений
            else:
                if(int(word[1]) not in index_dict[normal_form]):
                    index_dict[normal_form].append(int(word[1]))
                if(word[0] not in lem_dict[normal_form]):
                    lem_dict[normal_form].append(word[0].strip())
            if (normal_form not in token_count_dict.keys()):
                token_count_dict[normal_form] = {}
                token_count_dict[normal_form][int(word[1])] = 1
            else:
                if (int(word[1]) not in token_count_dict[normal_form].keys()):
                    token_count_dict[normal_form][int(word[1])] = 1
                else:
                    token_count_dict[normal_form][int(word[1])] += 1
                
    for word in eng_words:
        # получаем начальную форму слова
        normal_form = get_normal_form_eng(word[0])
        if normal_form:
            # если такое слово еще не встречалось,создаем ключ с нормальной формой и помещаем само слово как значение
            if normal_form not in lem_dict.keys():
                lem_dict[normal_form] = [word[0].strip()]
                index_dict[normal_form] = [int(word[1])]
            # если такое слово встречалось ранее, добавляем его вариацию в список значений
            else:
                if(int(word[1]) not in index_dict[normal_form]):
                    index_dict[normal_form].append(int(word[1]))
                if(word[0] not in lem_dict[normal_form]):
                    lem_dict[normal_form].append(word[0].strip())
            if (normal_form not in token_count_dict.keys()):
                token_count_dict[normal_form] = {}
                token_count_dict[normal_form][int(word[1])] = 1
            else:
                if (int(word[1]) not in token_count_dict[normal_form].keys()):
                    token_count_dict[normal_form][int(word[1])] = 1
                else:
                    token_count_dict[normal_form][int(word[1])] += 1
    
    for key in index_dict.keys():
        index_dict[key].sort()
                
    tf_dict = {}
    for token in token_count_dict.keys():
        tf_dict[token] = {}
        for doc in token_count_dict[token].keys():
            token_count =  token_count_dict[token][doc]
            total_words_count = docs_dict[doc]
            tf = token_count / total_words_count
            tf_dict[token][doc] = tf
            print(token, doc, token_count, total_words_count, tf)
    
    return(lem_dict, index_dict, tf_dict)

def get_normal_form_rus(word):
    return analyzer.parse(word)[0].normal_form
    
def get_normal_form_eng(word):
    return lemmatizer.lemmatize(word)

# записываем полученные результаты в формате:
# "начальная форма слова: токен токен ..."
# знак ":" служит разделителем между ключом и значениями
def save_lemmatized_tokens_to_file(lem_dict):
    file = open("lemmatized_tokens.txt", "w", encoding="utf-8")
    for word, tokens in lem_dict.items():
        file.write(f"{word}:")
        [file.write(f" {tok}") for tok in set(tokens)]
        file.write("\n")
    file.close()
    
def save_inverted_index_to_file(index_dict):
    file = open("inverted_index.txt", "w", encoding="utf-8")
    for word, indexes in index_dict.items():
        file.write(f"{word}")
        [file.write(f" {index}") for index in set(indexes)]
        file.write("\n")
    file.close()
    
def save_tf_idf_to_file(tf_idf_dict):
    file = open("tf_idf.txt", "w", encoding="utf-8")
    for token in tf_idf_dict.keys():
        for doc in tf_idf_dict[token].keys():
            file.write(f"{token} {doc} {tf_idf_dict[token][doc][0]} {tf_idf_dict[token][doc][1]}")
            file.write("\n")
    file.close()
    
def boolean_search(text, index_dict):
    # убираем знаки препинания
    pattern = r"[^\w]"
    text = re.sub(pattern, " ", text)
    words = text.split()
    words = filter_words(words)
    
    rus_words = [word for word in filter(rus_filter.match, words)]
    eng_words = [word for word in filter(eng_filter.match, words)]
    
    normal_words = []
    
    for word in rus_words:
        normal_words.append(get_normal_form_rus(word))
    for word in eng_words:
        normal_words.append(get_normal_form_eng(word))
    
    page_numbers = []
    for word in normal_words:
        if(word in index_dict.keys()):
            page_numbers.extend(index_dict[word])
    page_numbers.sort()
    return page_numbers

def idf(docs_count, index_dict):
    idf_dict = {}
    for token in index_dict.keys():
        idf = math.log(docs_count / len(index_dict[token]))
        idf_dict[token] = idf
    return idf_dict

def tf_idf(tf_dict, idf_dict):
    tf_idf_dict = {}
    for token in tf_dict.keys():
        tf_idf_dict[token] = {}
        for doc in tf_dict[token].keys():
            tf_idf = tf_dict[token][doc] * idf_dict[token]
            tf_idf_dict[token][doc] = [idf_dict[token], tf_idf]
    return tf_idf_dict

In [3]:
docs_dict = tokenize()
print(docs_dict)

{1: 74, 2: 1050, 3: 898, 4: 849, 5: 364, 6: 676, 7: 754, 8: 1073, 9: 894, 10: 2036, 11: 644, 12: 1166, 13: 940, 14: 1376, 15: 804, 16: 72, 17: 2036, 18: 602, 19: 344, 20: 740, 21: 1908, 22: 685, 23: 1022, 24: 421, 25: 343, 26: 512, 27: 1728, 28: 776, 29: 774, 30: 1005, 31: 1490, 32: 859, 33: 661, 34: 738, 35: 865, 36: 255, 37: 672, 38: 1945, 39: 861, 40: 713, 41: 852, 42: 906, 43: 759, 44: 798, 45: 1499, 46: 1402, 47: 761, 48: 226, 49: 1491, 50: 606, 51: 781, 52: 350, 53: 954, 54: 244, 55: 981, 56: 748, 57: 682, 58: 1252, 59: 2384, 60: 757, 61: 706, 62: 930, 63: 1067, 64: 1821, 65: 347, 66: 952, 67: 1384, 68: 766, 69: 633, 70: 430, 71: 1344, 72: 178, 73: 867, 74: 100, 75: 865, 76: 223, 77: 804, 78: 1010, 79: 678, 80: 1493, 81: 678, 82: 1030, 83: 733, 84: 680, 85: 638, 86: 731, 87: 216, 88: 64, 89: 32, 90: 684, 91: 1091, 92: 912, 93: 588, 94: 1007, 95: 540, 96: 804, 97: 738, 98: 1109, 99: 142, 100: 874, 101: 651, 102: 329, 103: 458, 104: 622, 105: 60, 106: 422, 107: 165, 108: 1125, 109:

In [4]:
lem_dict, index_dict, tf_dict = lemmatize(docs_dict)
print(tf_dict)

учебник 1 2 74 0.02702702702702703
учебник 15 1 804 0.0012437810945273632
учебник 67 1 1384 0.000722543352601156
учебник 103 1 458 0.002183406113537118
начинающий 1 2 74 0.02702702702702703
начинающий 86 2 731 0.0027359781121751026
начинающий 99 1 142 0.007042253521126761
продвинуть 1 2 74 0.02702702702702703
продвинуть 86 2 731 0.0027359781121751026
поиск 1 1 74 0.013513513513513514
поиск 2 1 1050 0.0009523809523809524
поиск 5 1 364 0.0027472527472527475
поиск 7 1 754 0.001326259946949602
поиск 8 1 1073 0.0009319664492078285
поиск 10 1 2036 0.0004911591355599214
поиск 12 1 1166 0.0008576329331046312
поиск 13 1 940 0.0010638297872340426
поиск 15 1 804 0.0012437810945273632
поиск 16 1 72 0.013888888888888888
поиск 17 1 2036 0.0004911591355599214
поиск 20 1 740 0.0013513513513513514
поиск 21 1 1908 0.0005241090146750524
поиск 22 1 685 0.00145985401459854
поиск 23 1 1022 0.0009784735812133072
поиск 24 1 421 0.0023752969121140144
поиск 28 1 776 0.001288659793814433
поиск 29 3 774 0.0038759

различный 39 1 861 0.0011614401858304297
различный 40 1 713 0.001402524544179523
различный 41 1 852 0.0011737089201877935
различный 42 1 906 0.0011037527593818985
различный 43 2 759 0.002635046113306983
различный 44 1 798 0.0012531328320802004
различный 45 1 1499 0.00066711140760507
различный 46 1 1402 0.0007132667617689016
различный 47 1 761 0.001314060446780552
различный 48 1 226 0.004424778761061947
различный 49 1 1491 0.0006706908115358819
различный 50 1 606 0.0016501650165016502
различный 51 1 781 0.0012804097311139564
различный 52 1 350 0.002857142857142857
различный 53 1 954 0.0010482180293501049
различный 55 1 981 0.0010193679918450561
различный 56 1 748 0.001336898395721925
различный 57 1 682 0.001466275659824047
различный 58 2 1252 0.001597444089456869
различный 59 1 2384 0.00041946308724832214
различный 60 4 757 0.005284015852047556
различный 61 1 706 0.00141643059490085
различный 62 1 930 0.001075268817204301
различный 63 4 1067 0.0037488284910965324
различный 64 1 1821 0.0

иметь 32 2 859 0.002328288707799767
иметь 34 2 738 0.0027100271002710027
иметь 35 2 865 0.0023121387283236996
иметь 36 1 255 0.00392156862745098
иметь 37 1 672 0.001488095238095238
иметь 38 5 1945 0.002570694087403599
иметь 39 1 861 0.0011614401858304297
иметь 40 1 713 0.001402524544179523
иметь 43 2 759 0.002635046113306983
иметь 45 1 1499 0.00066711140760507
иметь 47 1 761 0.001314060446780552
иметь 49 2 1491 0.0013413816230717639
иметь 51 1 781 0.0012804097311139564
иметь 52 2 350 0.005714285714285714
иметь 54 1 244 0.004098360655737705
иметь 55 1 981 0.0010193679918450561
иметь 56 1 748 0.001336898395721925
иметь 58 1 1252 0.0007987220447284345
иметь 59 4 2384 0.0016778523489932886
иметь 60 1 757 0.001321003963011889
иметь 61 1 706 0.00141643059490085
иметь 62 1 930 0.001075268817204301
иметь 63 1 1067 0.0009372071227741331
иметь 66 1 952 0.0010504201680672268
иметь 67 1 1384 0.000722543352601156
иметь 68 8 766 0.010443864229765013
иметь 70 1 430 0.002325581395348837
иметь 72 1 178

просто 3 1 898 0.0011135857461024498
просто 4 1 849 0.001177856301531213
просто 5 1 364 0.0027472527472527475
просто 6 1 676 0.0014792899408284023
просто 7 6 754 0.007957559681697613
просто 8 6 1073 0.005591798695246971
просто 9 1 894 0.0011185682326621924
просто 10 11 2036 0.0054027504911591355
просто 11 1 644 0.0015527950310559005
просто 12 6 1166 0.005145797598627788
просто 13 3 940 0.0031914893617021275
просто 14 1 1376 0.0007267441860465116
просто 15 2 804 0.0024875621890547263
просто 16 1 72 0.013888888888888888
просто 17 11 2036 0.0054027504911591355
просто 18 1 602 0.0016611295681063123
просто 19 1 344 0.0029069767441860465
просто 20 4 740 0.005405405405405406
просто 21 4 1908 0.0020964360587002098
просто 22 2 685 0.00291970802919708
просто 23 4 1022 0.003913894324853229
просто 24 2 421 0.004750593824228029
просто 25 1 343 0.0029154518950437317
просто 26 1 512 0.001953125
просто 27 1 1728 0.0005787037037037037
просто 28 4 776 0.005154639175257732
просто 29 2 774 0.0025839793281

в 91 12 1091 0.010999083409715857
в 93 4 588 0.006802721088435374
в 95 10 540 0.018518518518518517
в 98 9 1109 0.008115419296663661
в 99 2 142 0.014084507042253521
в 100 8 874 0.009153318077803204
в 101 16 651 0.02457757296466974
в 102 17 329 0.05167173252279635
в 103 5 458 0.010917030567685589
в 104 32 622 0.05144694533762058
в 105 2 60 0.03333333333333333
в 106 14 422 0.03317535545023697
в 107 2 165 0.012121212121212121
в 108 7 1125 0.006222222222222222
в 110 10 873 0.011454753722794959
публиковать 1 1 74 0.013513513513513514
публиковать 2 1 1050 0.0009523809523809524
публиковать 5 1 364 0.0027472527472527475
публиковать 7 1 754 0.001326259946949602
публиковать 8 1 1073 0.0009319664492078285
публиковать 10 1 2036 0.0004911591355599214
публиковать 12 1 1166 0.0008576329331046312
публиковать 13 1 940 0.0010638297872340426
публиковать 15 1 804 0.0012437810945273632
публиковать 16 1 72 0.013888888888888888
публиковать 17 1 2036 0.0004911591355599214
публиковать 20 1 740 0.001351351351351

...

btnbottomtext 43 2 759 0.002635046113306983
btnbottomtext 90 2 684 0.0029239766081871343
lltopcolor 43 2 759 0.002635046113306983
lltopcolor 90 2 684 0.0029239766081871343
llbottomcolor 43 2 759 0.002635046113306983
llbottomcolor 90 2 684 0.0029239766081871343
setbackgroundresource 43 1 759 0.0013175230566534915
setbackgroundresource 90 1 684 0.0014619883040935672
illustrates 44 1 798 0.0012531328320802004
bound 44 1 798 0.0012531328320802004
bound 53 1 954 0.0010482180293501049
bound 85 1 638 0.001567398119122257
bound 92 1 912 0.0010964912280701754
illustrating 44 1 798 0.0012531328320802004
news 44 3 798 0.0037593984962406013
notifies 44 1 798 0.0012531328320802004
capable 44 1 798 0.0012531328320802004
capable 85 1 638 0.001567398119122257
okay 44 1 798 0.0012531328320802004
dependning 44 1 798 0.0012531328320802004
mistake 44 1 798 0.0012531328320802004
mistake 64 2 1821 0.001098297638660077
recognized 44 1 798 0.0012531328320802004
figured 44 1 798 0.0012531328320802004
developed

droidcon 101 1 651 0.0015360983102918587
berlin 52 1 350 0.002857142857142857
berlin 101 1 651 0.0015360983102918587
support 52 5 350 0.014285714285714285
support 59 1 2384 0.00041946308724832214
support 96 1 804 0.0012437810945273632
support 100 1 874 0.0011441647597254005
support 108 2 1125 0.0017777777777777779
appcompat 52 1 350 0.002857142857142857
cardview 52 1 350 0.002857142857142857
execution 53 1 954 0.0010482180293501049
accepts 53 1 954 0.0010482180293501049
determines 53 1 954 0.0010482180293501049
viewgroups 53 1 954 0.0010482180293501049
viewgroups 57 2 682 0.002932551319648094
relevant 53 1 954 0.0010482180293501049
binding 53 1 954 0.0010482180293501049
converted 53 1 954 0.0010482180293501049
alright 53 1 954 0.0010482180293501049
alright 71 1 1344 0.000744047619047619
command 53 1 954 0.0010482180293501049
stay 53 1 954 0.0010482180293501049
bind 53 1 954 0.0010482180293501049
differs 53 1 954 0.0010482180293501049
differs 64 1 1821 0.0005491488193300384
refactor 53 

ui 101 1 651 0.0015360983102918587
ui 106 1 422 0.002369668246445498
ui 110 1 873 0.001145475372279496
databasae 64 1 1821 0.0005491488193300384
ivan 64 6 1821 0.0032948929159802307
maria 64 6 1821 0.0032948929159802307
petr 64 6 1821 0.0032948929159802307
anton 64 6 1821 0.0032948929159802307
dasha 64 6 1821 0.0032948929159802307
boris 64 6 1821 0.0032948929159802307
kostja 64 6 1821 0.0032948929159802307
igor 64 6 1821 0.0032948929159802307
accountant 64 11 1821 0.0060406370126304225
director 64 8 1821 0.004393190554640308
security 64 8 1821 0.004393190554640308
security 96 1 804 0.0012437810945273632
grouped 64 1 1821 0.0005491488193300384
selects 64 1 1821 0.0005491488193300384
rowsid 64 3 1821 0.0016474464579901153
fine 64 1 1821 0.0005491488193300384
fine 73 1 867 0.0011534025374855825
fine 75 1 865 0.0011560693641618498
fine 77 1 804 0.0012437810945273632
fine 83 1 733 0.001364256480218281
fine 94 1 1007 0.0009930486593843098
fine 96 1 804 0.0012437810945273632
designing 64 1 18

alayout 78 1 1010 0.0009900990099009901
alayout 92 1 912 0.0010964912280701754
surname 79 2 678 0.0029498525073746312
mainacitivty 79 1 678 0.0014749262536873156
acquainted 79 1 678 0.0014749262536873156
variety 79 1 678 0.0014749262536873156
equipped 79 1 678 0.0014749262536873156
incoming 79 1 678 0.0014749262536873156
whatever 79 1 678 0.0014749262536873156
receiving 79 1 678 0.0014749262536873156
ourself 81 1 678 0.0014749262536873156
uncommon 81 1 678 0.0014749262536873156
supporting 81 1 678 0.0014749262536873156
suggests 81 1 678 0.0014749262536873156
intentfilters 81 1 678 0.0014749262536873156
alternative 81 1 678 0.0014749262536873156
webpage 81 1 678 0.0014749262536873156
damn 81 1 678 0.0014749262536873156
random 81 1 678 0.0014749262536873156
dark 81 1 678 0.0014749262536873156
rubbish 81 1 678 0.0014749262536873156
ethic 81 1 678 0.0014749262536873156
adequacy 81 1 678 0.0014749262536873156
sense 81 1 678 0.0014749262536873156
dpвокруг 82 1 1030 0.000970873786407767
dpотс

sample 96 2 804 0.0024875621890547263
applicationsgoogle 96 1 804 0.0012437810945273632
weigh 96 1 804 0.0012437810945273632
gigabyte 96 1 804 0.0012437810945273632
reject 96 1 804 0.0012437810945273632
indicator 96 1 804 0.0012437810945273632
phrase 96 1 804 0.0012437810945273632
weighted 96 1 804 0.0012437810945273632
gb 96 1 804 0.0012437810945273632
performing 96 1 804 0.0012437810945273632
maybe 96 1 804 0.0012437810945273632
dull 96 1 804 0.0012437810945273632
boring 96 1 804 0.0012437810945273632
fun 96 1 804 0.0012437810945273632
googling 96 1 804 0.0012437810945273632
facing 96 1 804 0.0012437810945273632
forum 96 1 804 0.0012437810945273632
setup 96 1 804 0.0012437810945273632
accidentally 96 1 804 0.0012437810945273632
popped 96 1 804 0.0012437810945273632
informs 96 1 804 0.0012437810945273632
informs 97 1 738 0.0013550135501355014
product 96 1 804 0.0012437810945273632
greedy 96 1 804 0.0012437810945273632
reached 97 1 738 0.0013550135501355014
included 97 1 738 0.00135501

In [5]:
docs_count = len(docs_dict.keys())
idf_dict = idf(docs_count, index_dict)

tf_idf_dict = tf_idf(tf_dict, idf_dict)

In [6]:
save_lemmatized_tokens_to_file(lem_dict)
save_inverted_index_to_file(index_dict)
save_tf_idf_to_file(tf_idf_dict)

In [7]:
text = "Опишем, как решается задача."
page_numbers = boolean_search(text, index_dict)
print(page_numbers)

[10, 12, 15, 17, 31, 32, 47, 49, 52, 59, 66, 72, 78, 80, 80, 101, 103, 104, 110]
